# RFZUpscale

**RFZUpscale** memanfaatkan **GPU Nvidia T4** di **Google Colab** untuk **meningkatkan kualitas gambar secara otomatis** dengan detail yang lebih tajam dan kejernihan yang lebih baik. Dengan dukungan GPU Nvidia T4.

© 2025 RFZUpscale by [RizzyFuzz](http://www.rizzy.eu.org)

In [ ]:
#@title **Install Package**
!pip uninstall -y huggingface_hub
!pip install huggingface_hub==0.25.1
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

from IPython.display import clear_output
clear_output()
print("✅ Semua paket telah diinstall.")

In [ ]:
#@title **Create model & Upload with Save to Drive or Download ZIP**
from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch
import os
import sys
import tarfile
import shutil
from io import BytesIO
from google.colab import files
from google.colab import drive

# ===== SETUP FOLDER =====
upload_folder = 'inputs'
result_folder = 'results'
os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# ===== DETEKSI CUDA =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('✅ Device:', device)

# ===== PILIH SKALA MODEL =====
model_scale = "4" #@param ["2", "4", "8"] {allow-input: false}
model_path = f'weights/RealESRGAN_x{model_scale}.pth'

# ===== CEK FILE MODEL =====
if not os.path.exists(model_path):
    raise FileNotFoundError(f"❌ Model file '{model_path}' tidak ditemukan. Pastikan file ada di folder 'weights'.")

# ===== LOAD MODEL =====
model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(model_path)
print(f"✅ Model x{model_scale} berhasil dimuat.\n")

# ===== FORMAT TAR SUPPORT =====
IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = BytesIO(buff.read())
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(fp.getvalue())
    fp.seek(0)
    return img_tar_info, fp

def process_tar(path_to_tar):
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join(result_folder, os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, Processing {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'Unable to open file {member.name}, skipping')
            continue

        img_sr = model.predict(np.array(img_lr))
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()
    print(f'✅ Finished! Archive saved to {result_tar_path}')

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join(result_folder, '[HD] ' + os.path.basename(filename))
        image = Image.open(filename).convert('RGB')
        sr_image = model.predict(np.array(image))
        sr_image.save(result_image_path)
        print(f'✅ Finished processing {filename}\n')

        # Remove original file
        os.remove(filename)

# ===== UPLOAD & PROSES =====
try:
    print('📁 Pilih gambar/tar file untuk di-enhance:\n')
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f'✨ Enhance: {filename}')
        process_input(filename)

    # Input validasi pilihan simpan hasil
    while True:
        pilihan = input(
            "\nPilih opsi simpan hasil:\n"
            "1. Simpan ke Google Drive (folder RFZUpscale)\n"
            "2. Download ZIP hasil\n"
            "Masukkan nomor pilihan (1/2) atau tekan Enter untuk default (1): "
        ).strip()

        if pilihan == '':
            pilihan = '1'  # default ke Google Drive

        if pilihan not in ('1', '2'):
            print("⚠️ Masukkan angka 1 atau 2 saja!")
        else:
            break

    if pilihan == '1':
        print("⚡ Mounting Google Drive...")
        drive.mount('/content/drive')

        drive_save_folder = '/content/drive/MyDrive/RFZUpscale'
        if not os.path.exists(drive_save_folder):
            os.makedirs(drive_save_folder)

        for f in os.listdir(result_folder):
            shutil.copy(os.path.join(result_folder, f), drive_save_folder)

        print(f"✅ Semua hasil disimpan di Google Drive folder:\n{drive_save_folder}")

    elif pilihan == '2':
        shutil.make_archive('download', 'zip', result_folder)
        files.download('download.zip')
        print("✅ File ZIP siap diunduh.")

except KeyboardInterrupt:
    print('\n❌ Upload dibatalkan.')
    sys.exit()

finally:
    shutil.rmtree(upload_folder)
    shutil.rmtree(result_folder)